pip install selenium

pip install xeus-python

Código pra baixar múltiplos arquivos exceto os de linha grande

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time


# Inciando a seção e abrindo a tabela de Cataguases
driver = webdriver.Chrome()
driver.maximize_window()
original_window = driver.current_window_handle


driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

select_element = driver.find_element(By.ID,'uregi')
select_object = Select(select_element)

select_object.select_by_index(28)

search_button = driver.find_element(By.ID, "ok")
search_button.click()


#Pegando o arquivo e as infos de cada linha e salvando em um dicionário

list_info = []
linha = 2
scroll = 0

while True:
    
    #WebDriverWait(driver).until(lambda d: d.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]))
    #expected_conditions.presence_of_element_located(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}])
    
    #Pegando as informações de cada linha e colocando em uma lista
    for i in range(1,6):
        list_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]").text)
        #print(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[{i}]").text)
    print(list_info)
    
    
    #Clicando na imagem do arquivo
    driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[6]").click()
    
    #WebDriverWait(driver, 10).until(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]"))
    
    #WebDriverWait(driver, 10).until(lambda d: d.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]"))
    
    #Tentando arrastar pro lado pra corrigir o problema do IndexError: list index out of range
    #driver.execute_script("window.scrollTo(0, document.body.scrollWeight);")
    #ActionChains(driver).click_and_hold(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]")).move_by_offset(2000 , 0).release().perform()
    
    
    try:
        driver.switch_to.window(driver.window_handles[1])
    
    except:
        #Arrastar pra direita
        #driver.find_element(By.TAG_NAME, f"html").send_keys(Keys.ARROW_RIGHT)
        
        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
        print(scroll)
        driver.execute_script(f"window.scrollTo(500,{scroll})") #x,y
        linha = linha + 1
        
        list_info.clear()
    
    else:
                
        driver.find_element(By.LINK_TEXT, "Download").click()
        
        #Função que cria o nome do arquivo com base nas infos da linha
        formatar_string(list_info)
        
        #Close the tab or window
        driver.close()

        #Switch back to the old tab or window
        driver.switch_to.window(original_window)

        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
        driver.execute_script(f"window.scrollTo(500,{scroll})")
        
        linha = linha + 1
        list_info.clear()

        


Código mais simples pra trabalhar com a parte dos arquivos

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from pathlib import Path
import time
import os
import shutil


# Inciando a seção e abrindo a tabela de Cataguases
driver = webdriver.Chrome()
driver.maximize_window()
original_window = driver.current_window_handle


driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

select_element = driver.find_element(By.ID,'uregi')
select_object = Select(select_element)

select_object.select_by_index(5) #Cataguases

search_button = driver.find_element(By.ID, "ok")
search_button.click()


list_info = []
linha = 2


#Pegando as informações de cada linha e colocando em uma lista
for i in range(1,6):
    list_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]").text)

print(list_info)

#Gerando o nome correto
formatar_string(list_info)


#Baixando o arquivo da linha
driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[6]").click()

driver.switch_to.window(driver.window_handles[1])

driver.find_element(By.LINK_TEXT, "Download").click()

driver.close()

driver.switch_to.window(original_window)

# Retorna o caminho com o nome do arquivo mais recente baixado
pegar_nome_original()


renomear_e_mover_arquivo(formatar_string(list_info), pegar_nome_original(), list_info[1])




time.sleep(30)
driver.quit()


Tentando descobrir como pegar o arquivo de linhas grandes

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import time


# Inciando a seção e abrindo a tabela de Teófilo Otoni
driver = webdriver.Chrome()
driver.maximize_window()
original_window = driver.current_window_handle


driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

select_element = driver.find_element(By.ID,'uregi')
select_object = Select(select_element)

select_object.select_by_index(28) #Teófilo Otoni

search_button = driver.find_element(By.ID, "ok")
search_button.click()


list_info = []
linha = 2
scroll = 0


while True:

    #Pegando as informações de cada linha e colocando em uma lista
    for i in range(1,6):
        list_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]").text)

    print(list_info)
    
    #Baixando o arquivo da linha
    #driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[6]/a/img").click()

    driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[6]/a/img").click()

    #cabecalho = driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[2]/td[6]").text
    #print(cabecalho)
    

    try:
         driver.switch_to.window(driver.window_handles[1])

    except:
        print("Deu errado de novo...")

        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
        print(scroll)
        driver.execute_script(f"window.scrollTo(500,{scroll})") #x,y
        
        linha = linha + 1
        list_info.clear()

        driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[6]/a/img").click()

    else:
        
        #time.sleep(2) # Só pra testar o porquê de não se baixar todos os arquivos
        
        try:
            driver.find_element(By.LINK_TEXT, "Download").click()
        
        except:
            print("Não consegui clicar no link ahhhhh")
            assert "Não deu certo"
        
        else:
            driver.close()

            driver.switch_to.window(original_window)

            scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
            driver.execute_script(f"window.scrollTo(500,{scroll})")

            linha = linha + 1
            list_info.clear()

            print(f"Eu devia ter pego {linha - 1} arquivos")

time.sleep(30)
driver.quit()
pass


### Função pra transformar a lista de infos na string a se renomear os arquivos

In [1]:
def formatar_string(list_info):
 
    #2.3_Almenara_Vanderlan Dias_2022.01.19
    #nºdameta_Município_Primeironome Últimonome_2022.mês.dia.pdf
    
    meta = list_info[3][17:20]
    município = list_info[1]
    cliente = list_info[4].split()
    data = list_info[5].split('/')
    
    if(meta == "2.3"):
        nome_do_arquivo = meta + "_" + município + "_" + cliente[0] + " " + cliente[-1] + "_" + data[2] + "." + data[1] + "." + data[0] + ".pdf"
    else:
        nome_do_arquivo = meta + "_" + município + "_" + cliente[0] + " " + cliente[1] + " et al" + "_" + data[2] + "." + data[1] + "." + data[0] + ".pdf"
            
    return nome_do_arquivo

### Função pra pegar o nome original do arquivo

In [2]:
def pegar_nome_original():
    
    time.sleep(3) # Para dar tempo de baixar o arquivo, o tempo pode ser maior ou menor dependendo da internet
    # Talvez eu possa tirar esse tempo, não sei
    """
        Tem formas avançadas de determinar se o arquivo foi baixado. Aqui alguns links
        https://stackoverflow.com/questions/34338897/python-selenium-find-out-when-a-download-has-completed/34338926#34338926
        https://stackoverflow.com/questions/21746750/check-and-wait-until-a-file-exists-to-read-it
    """
        
    data_criacao = lambda f: f.stat().st_ctime
    # data_modificacao = lambda f: f.stat().st_mtime

    directory = Path(r'C:\Users\61445\Downloads')
    files = directory.glob('*.pdf')
    nome_original = sorted(files, key=data_criacao, reverse=True)
    # Alterei de sorted_files pra nome_original só pq vou precisar do caminho todo na renomear_arquivo
    
    """
    Essa função vai pegar todos os pdfs da pasta e colocar o caminho deles em uma lista(nome_original) da qual eu vou pegar o
    primeiro termo que seria o mais recente criado. Então por tudo que é mais sagrado, que não tenha outros arquivos pdfs de
    outra natureza nessa pasta senão vai renomear e mover ele.
    """
    
    # print(nome_original)
    
    if not nome_original:
        nome_original.append("Alguma coisa")
        
    # print(nome_original)
    # print("\n")
    return nome_original[0] #Tem o endereço completo ou a expressão "Alguma coisa"
    
    

### Renomeando e movendo o arquivo baixado

In [3]:
def renomear_e_mover_arquivo(nome_formatado, nome_original, SRE):
    #print(nome_formatado)
    #print(nome_original)
    
    path = 'C:\\Users\\61445\\Downloads\\'
    #path = 'C:\\Users\\1234\\Downloads\\'
    meta = nome_formatado[:3]
    
    #nome_original = os.path.join(path, nome_original)
    #nome_formatado = os.path.join(path, nome_formatado)
    
    """
    Preciso de alguma forma de que se não tiver pdf na pasta de download (pq não baixou) ele me alertar
    """
    
    if(nome_original != "Alguma coisa"):
        os.rename(nome_original, nome_formatado)
        
        endereco_do_arquivo_a_ser_movido = path + nome_formatado
    
        diretório_de_destino = path + "Evidencias das Uregis - Transição\\" + SRE + "\\" + meta
        #print(diretório_de_destino)

        shutil.move(endereco_do_arquivo_a_ser_movido,diretório_de_destino)
        # Precisa que tenha a pasta da SRE já criada, do contrário ele renomeia o arquivo para o nome da pasta e fica até engraçado

        #Para apagar o arquivo, é só para testes
        #time.sleep(5)
        #os.remove(diretório_de_destino + "\\" + nome_formatado)
    else:
        #raise Exception("O renome não funcionou")
        pass # Não vai fazer nada, não tem arquivo ali pra renomear
        """ Talvez colocar aqui algo para apagar todo pdf que estiver na pasta de download (caso ele tenha sido baixado 
        dps que a pegar_nome_original foi acionada e não pegou esse arquivo, se isso acontecesse, o próximo que não 
        baixasse e por já ter um ali, não ia retornar "Alguma coisa" e ele que iria ser movido pra outra pasta, causando problema,
        um arquivo no lugar de outro que deveria estar em branco = confusão, melhor evitar).
        Mas pensando bem isso deveria vir antes de baixar o arquivo... tipo, antes do clique no 'download'
        Mas vai-se conferir de toda forma, melhor esperar isso acontecer antes de pensar em uma remediação, mas deixo aqui o 
        registro da possibilidade disso acontecer."""
        
        """ Colocar aqui pra adicionar que o arquivo não foi encontrado/baixado/renomeado/movido no txt feito nessa linha. 
        Abrir um open do arquivo e colocar a mensagem"""

### Gerando a lista que irá para o txt

In [4]:
def gera_lista_txt(list_info):
    
    meta = list_info[3][17:20]
    lista_para_txt = list_info.copy()
    
    lista_de_nomes = lista_para_txt[4].split()
    lista_de_nomes = lista_de_nomes[:2]
    
    
    if(meta != "2.3"):
        lista_de_nomes.append("et al")
        
    nomes_em_string = ' '.join(lista_de_nomes)
    
    lista_para_txt[4] = nomes_em_string
    
    #print("lista_para_txt", lista_para_txt)
    
    return lista_para_txt

### Gerando o arquivo de txt com as infos de cada Uregi

In [5]:
def arquivo_txt(infos, nome_do_arquivo, uregi):
    
    #path = 'C:\\Users\\1234\\Downloads'
    path = 'C:\\Users\\61445\\Downloads\\'
    
    # Convertendo a lista em string
    infos_str = '; '.join(infos)
    #print(infos_str)
    
    with open(fr'{path}\\{uregi[12:]}.txt', 'a') as file:
        file.write(infos_str)
        file.write("\n")
        file.write(nome_do_arquivo)
        file.write("\n\n")

## Código completo

In [12]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from pathlib import Path
import time
import os
import shutil


uregi = "Regional de Teófilo Otoni"
list_info = [] #lista que conterá todas as informações de cada linha da tabela
linha = 2
scroll = 0


# Inciando a seção e abrindo a tabela de Teófilo Otoni
driver = webdriver.Chrome()
driver.maximize_window()
original_window = driver.current_window_handle

driver.get("https://www.emater.mg.gov.br/portal.do?flagweb=novaintranet_pgn_pmater_educ_lista_uregi")

select_element = driver.find_element(By.ID,'uregi')
select_object = Select(select_element)

# Aqui é onde se coloca a Uregi que se deseja
select_object.select_by_visible_text(uregi)

search_button = driver.find_element(By.ID, "ok")
search_button.click()

"""
time.sleep(7)
scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha - 1}]").rect['height']
print(scroll) #Deu problema
#driver.execute_script(f"window.scrollTo(500,{scroll})")
"""

"""
try: Se lançar erro de não achar os dados (pode ser um termo qualquer tipo a data)

except:
    comando de for pra descer até a linha de interesse armazenando a altura dela na scroll
    pedir pra descer até lá

finally:
    O resto do programa a partir do while

"""

try:
    driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[6]/a/img").click()
except:
    for i in range(2,linha): #Nesse caso, desço até o fim da linha anterior abrindo espaço pra linha de interesse
        scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{i}]").rect['height']
    driver.execute_script(f"window.scrollTo(500,{scroll})")
else:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()
    driver.switch_to.window(original_window)

finally:    
    while True:

        #Pegando as informações de cada linha e colocando em uma lista
        list_info.append(f"Índice: {linha - 1}")
        for i in range(1,6):
            list_info.append(driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[{i}]").text)
        #print(list_info)


        # Clicando na imagem para abrir a aba de download
        driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]/td[6]/a/img").click()

        try:
             driver.switch_to.window(driver.window_handles[1])

        except:

            raise Exception("Não foi possível abrir a aba desse arquivo") #Só pra depuração

            scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
            driver.execute_script(f"window.scrollTo(500,{scroll})") #x,y

            linha = linha + 1
            list_info.clear()

        else:

            driver.find_element(By.LINK_TEXT, "Download").click()
            
            #time.sleep(1) #Potencial para resolver muitos problemas
            driver.close()

            driver.switch_to.window(original_window)


            # Irá gerar o txt com as informações caso dê algum problema na automação
            arquivo_txt(gera_lista_txt(list_info), formatar_string(list_info), uregi)
            """
                gera_lista_txt: 
                    Retorna a linha da tabela em formato de string (cortando uma parte dos nomes quando tiver muitos pra não ficar confuso.)
                formatar_string:
                    Retorna o nome que o arquivo deve ter conforme o padrão
                uregi:
                    O nome da Uregi escolhida que será o nome do arquivo txt
            """


            # Renomeia e move o arquivo pra sua pasta correta
            renomear_e_mover_arquivo(formatar_string(list_info), pegar_nome_original(), list_info[2])
            """
                formatar_string:
                    Retorna o nome que o arquivo deve ter conforme o padrão
                pegar_nome_original:
                    Retorna o endereço e nome do arquivo mais recente criado (ou seja, o arquivo das metas baixado)
                list_info[2]:
                    É apenas a SRE da cidade da linha analisada
            """

            # Eu peço pra que a tela desça o tamanho da altura da linha analizada para abrir caminho para que a próxima linha possa estar visível
            scroll += driver.find_element(By.XPATH, f"/html/body/section/div/div[1]/table/tbody/tr[{linha}]").rect['height']
            driver.execute_script(f"window.scrollTo(500,{scroll})")

            linha = linha + 1
            list_info.clear() #limpa a lista para o próximo turno

            print(f"Eu devia ter pego {linha - 2} arquivos")

Error: Destination path 'C:\Users\61445\Downloads\Evidencias das Uregis - Transição\Teófilo Otoni\2.1\2.1_Itambacuri_Karisma Rodrigues et al_2021.12.17.pdf' already exists